In [ ]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from chempy import ReactionSystem
from chempy.kinetics.ode import get_odesys
from chempy.kinetics.analysis import plot_reaction_contributions
%matplotlib inline

In [ ]:
# Rate constants choosen rather arbitrary for demonstration purposes
system_str = """
  2 Fe+2 +  H2O2 -> 2 Fe+3     + 2 OH-        ; 42
  2 Fe+3 +  H2O2 -> 2 Fe+2     +   O2  + 2 H+ ; 17
      H+ +   OH- ->   H2O                     ; %(kprot_H2O)s
             H2O ->   H+       +   OH-        ; %(kprot_H2O)s * %(Kw)s
   Fe+3  + 2 H2O ->   FeOOH(s) + 3 H+         ; 1
FeOOH(s) + 3  H+ ->   Fe+3     + 2 H2O        ; 2.5
            H2O2 ->   H+       + HO2-         ; %(kprot_H2O2)s * %(Ka_H2O2)s
      H+ +  HO2- ->   H2O2                    ; %(kprot_H2O2)s
    H2O2 +   OH- ->   HO2-     + H2O          ; %(kdepr_H2O2)s
    HO2- +   H2O ->   H2O2     + OH-          ; %(kdepr_H2O2)s / %(Ka_H2O2)s * %(Kw)s
""" % dict(Kw='1e-14', kprot_H2O='1e10', Ka_H2O2='10**-11.65', kprot_H2O2='5e10', kdepr_H2O2='1.3e10')

In [ ]:
rsys = ReactionSystem.from_string(system_str)  # "[H2O]" = 1.0 (actually 55.4 at RT)
odesys, extra = get_odesys(rsys)
rsys

In [ ]:
tout = sorted(np.concatenate((np.linspace(0, 1003), np.logspace(-8, 3))))
c0 = defaultdict(float, {'Fe+2': 0.05, 'H2O2': 0.1, 'H2O': 1.0, 'H+': 1e-7, 'OH-': 1e-7})
result = odesys.integrate(tout, c0, atol=1e-12, rtol=1e-14)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 6))
result.plot(names=[k for k in rsys.substances if k != 'H2O'], ax=axes[0])
axes[0].legend(loc='best', prop={'size': 10}); _ = axes[0].set_xlabel('Time'); _ = axes[0].set_ylabel('Concentration')
result.plot(names=[k for k in rsys.substances if k != 'H2O'], ax=axes[1], xscale='log', yscale='log')
axes[1].legend(loc='best', prop={'size': 10}); _ = axes[1].set_xlabel('Time'); _ = axes[1].set_ylabel('Concentration')
result.plot(deriv=True, names=[k for k in rsys.substances if k != 'H2O'], ax=axes[2], xscale='log'); 
axes[2].set_yscale('symlog', linthreshy=1e-9)
axes[2].legend(loc='best', prop={'size': 10}); _ = axes[2].set_xlabel('Time'); _ = axes[2].set_ylabel('dC/dt')
result.plot_invariant_violations(ax=axes[3], xscale='log')
axes[3].legend(loc='best', prop={'size': 10}); _ = axes[3].set_xlabel('Time'); _ = axes[3].set_ylabel('Component')
plt.tight_layout()

In [ ]:
substance_keys = 'Fe+2 Fe+3 FeOOH(s) H2O2'.split()
selection = result.xout > 10
fig, axes = plt.subplots(len(substance_keys), 2, figsize=(16, 4*len(substance_keys)))
plot_reaction_contributions(result.xout, result.yout, extra['rate_exprs_cb'], rsys, substance_keys, axes=axes[:, 0])
plot_reaction_contributions(result.xout[selection], result.yout[selection, :], extra['rate_exprs_cb'], rsys,
                            substance_keys, axes=axes[:, 1], relative=True, xscale='linear', yscale='linear',
                            combine_equilibria=True)
plt.tight_layout()